In [1]:
import boto3

In [2]:
bucket_name = 'citizensalmon'
prefix = 'larson/raw-data/'

In [3]:
# create the necessary boto3 objects for interacting with S3
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

In [4]:
i = 0
total_object_size = 0

# iterate over all files under the desired prefix
for obj in bucket.objects.filter(Prefix = prefix):
    if obj.size == 0:
        continue
    # show basic info about each object
    print '%s: %s' % (obj.key, obj.size)
    # keep tally of file count and size
    i += 1
    total_object_size += obj.size
print '%s files found with total of %s bytes' % (i, total_object_size)

larson/raw-data/cjfas-2013-0502suppla.xlsx: 81357
larson/raw-data/cjfas-2013-0502supplb.docx: 79493
larson/raw-data/cjfas-2013-0502supplc.xlsx: 12252
larson/raw-data/cjfas-2013-0502suppld.xlsx: 12305
larson/raw-data/cjfas-2013-0502supple.docx: 25643
larson/raw-data/cjfas-2013-0502supplf.xlsx: 9156
larson/raw-data/cjfas-2013-0502suppli.fa: 2829761
larson/raw-data/genepop_western_alaska_chinook_RAD.txt: 14615158
8 files found with total of 17665125 bytes


In [ ]:
# function useful for streaming data from s3 without storing locally
def to_StringIO(info = None, s3 = None, local = False,
                bucket = None, key = None, file_name = None):
    # validate inputs
    if (info is None and (bucket is None or key is None)
        and file_name is None):
        raise ValueError('Must supply an info dictionary if all other'
                         'inputs are None')
    if info is None:
        info = {}
    if bucket is None:
        bucket = info.get('bucket', None)
    if key is None:
        key = info.get('key', None)
    if file_name is None:
        file_name = info.get('file', None)
    if (bucket is None or key is None) and file_name is None:
        raise ValueError('Must supply either a file_name or bucket/key pair')
    
    # stream from local file
    if local and file_name is not None:
        # open as binary if compressed
        r = 'rb' if file_name.endswith('gz') else 'r'
        with open(file_name, r) as fh:
            f = StringIO(fh.read())
            f.seek(0)
            return f
    
    # use default credential config if none provided
    if s3 is None:
        s3 = boto3.resource('s3')
    obj = s3.Object(bucket, key)
    f = StringIO(obj.get()['Body'].read())
    f.seek(0)
    return f